# [ EXPLORATION ] 15. 아이유팬이 좋아할 만한 다른 아티스트 찾기

✅추천시스템의 기본적인 원리와 구성을 파악하고 Last.fm 데이터 빛 MovieLes 데이터를 통해 간단하고 효과적인 아티스트 추천시스템과 영화 추천시스템을 구현해 본다.

---
### - 📖목차
* ✔️15-9. 프로젝트 - Movielens 영화 추천 실습
* ✔️회고 및 결론
* ✔️Reference(참고자료)

<br>



## ✔️15-9. 프로젝트 - Movielens 영화 추천 실습
---
### 1) 데이터 준비와 전처리

In [ ]:
# 모듈 로드
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# 경고 무시
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

In [ ]:
%%capture
!pip install implicit

In [ ]:
# 데이터 불러오기
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls '/content/gdrive/MyDrive/aiffel/ex_15/data'

movies.dat  ratings.dat  README  users.dat


In [ ]:
# 데이터 확인

rating_file_path= '/content/gdrive/MyDrive/aiffel/ex_15/data/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
# 3점 이상만 남기기
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [ ]:
# ratings 컬럼의 이름을 counts
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [ ]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [ ]:
# 영화 제목을 보기 위해 메타 데이터를 읽기
movie_file_path= '/content/gdrive/MyDrive/aiffel/ex_15/data/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
df1 = ratings.set_index("movie_id")
ratings2 = ratings.join(movies.set_index('movie_id'), on='movie_id')
using_cols = ['user_id', 'movie_id', 'counts', 'title']

ratings2 = ratings2[using_cols]
ratings2.head(10)

,user_id,movie_id,counts,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"
5,1,1197,3,"Princess Bride, The (1987)"
6,1,1287,5,Ben-Hur (1959)
7,1,2804,5,"Christmas Story, A (1983)"
8,1,594,4,Snow White and the Seven Dwarfs (1937)
9,1,919,4,"Wizard of Oz, The (1939)"


<br>

### 2) 분석해 봅시다.

- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [ ]:
# 영화 개수
ratings2['movie_id'].nunique()


3628

In [ ]:
# 유저 수
ratings2['user_id'].nunique()

6039

In [ ]:
movie_count = ratings2.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(50)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

<br>

### 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [ ]:
my_favorite = ['Men in Black (1997)','Matrix, The (1999)','Star Wars: Episode VI - Return of the Jedi (1983)','E.T. the Extra-Terrestrial (1982)','Toy Story (1995)']
my_favorite_id = ['2297','2434','2716','2102','2000']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정
my_movielist = pd.DataFrame({'user_id': ['buyel']*5, 'movie_id' : my_favorite_id,'counts':[5]*5, 'title': my_favorite})

if not ratings2.isin({'user_id':['buyel']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없을 시
    ratings2 = ratings2.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터 추가

ratings2.tail(10)    

,user_id,movie_id,counts,title
0,6039,2297,5,175
1,6039,2434,5,124
2,6039,2716,5,64
3,6039,2102,5,26
4,6039,2000,5,40
0,buyel,2297,5,Men in Black (1997)
1,buyel,2434,5,"Matrix, The (1999)"
2,buyel,2716,5,Star Wars: Episode VI - Return of the Jedi (1983)
3,buyel,2102,5,E.T. the Extra-Terrestrial (1982)
4,buyel,2000,5,Toy Story (1995)


In [ ]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings2['user_id'].unique()
movie_unique = ratings2['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자이다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [ ]:
# 인덱싱이 잘 되었는지 확인. 
print(user_to_idx['buyel'])
print(movie_to_idx['Men in Black (1997)'])

6040
3628


In [ ]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 에서 참고

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 보자
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거
temp_user_data = ratings2['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings2):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings2['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱
temp_movie_data = ratings2['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings2):
    print('title column indexing OK!!')
    ratings2['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings2

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,counts,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
0,6039,2297,5,175
1,6039,2434,5,124
2,6039,2716,5,64
3,6039,2102,5,26


<br>

### 4) CSR matrix를 직접 만들어 봅시다.

In [ ]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings2['user_id'].nunique()
num_movie = ratings2['title'].nunique()

csr_data = csr_matrix((ratings2.counts, (ratings2.user_id, ratings2.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

<br>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [ ]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [ ]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=25, dtype=np.float32)

In [ ]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose 해준다.)
#csr_data_transpose = csr_data.T
csr_data_transpose = csr_data

In [ ]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/25 [00:00<?, ?it/s]

<br>

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [ ]:
buyel, Matrix = user_to_idx['buyel'], movie_to_idx['Matrix, The (1999)']
buyel_vector, Matrix_vector = als_model.user_factorsbuyel], als_model.item_factors[Matrix]

In [ ]:
buyel_vector

array([-7.53163338e-01,  1.13992326e-01, -1.14833834e-02,  5.13385646e-02,
        7.28359759e-01, -2.99719483e-01, -3.55087556e-02,  3.49307537e-01,
        6.62303329e-01,  5.29432356e-01,  3.62202078e-02, -5.80792665e-01,
       -1.18047982e-01,  4.82324421e-01, -1.34663749e+00,  1.11428046e+00,
       -5.46255708e-02, -2.85410583e-01,  2.43565246e-01, -4.46852058e-01,
        8.80008280e-01, -7.38415897e-01, -1.88611537e-01,  5.64157188e-01,
       -1.84189186e-01,  3.40493649e-01,  6.43773735e-01, -2.94152826e-01,
        1.98749006e-01, -1.37997925e-01,  4.73932564e-01, -1.43922970e-01,
        4.79624867e-01,  1.71484053e-01, -4.75226268e-02, -3.41632545e-01,
       -1.51106879e-01, -8.88972133e-02,  6.68643057e-01,  6.80411384e-02,
        2.19610348e-01,  8.65211785e-02,  2.39156841e-04, -7.00243115e-01,
        6.49182618e-01,  8.31946254e-01, -3.64707202e-01, -8.71338844e-02,
       -5.78571856e-01,  2.86633551e-01,  2.74649620e-01, -2.02073693e-01,
        8.92164052e-01, -

In [ ]:
Matrix_vector

array([-0.01066333,  0.00472578,  0.00384912,  0.00248695,  0.03168683,
        0.02153509,  0.02414745,  0.03188241,  0.03334448,  0.02321646,
       -0.00242877, -0.02299661,  0.0183045 ,  0.03212208, -0.02829956,
        0.00365882, -0.00102274, -0.00970952,  0.004776  ,  0.00183218,
        0.0403115 , -0.02089067,  0.01530248,  0.01690111,  0.01301125,
        0.00410482,  0.02754636,  0.01953262,  0.02173669,  0.01561468,
        0.02619692,  0.01600741,  0.03891899,  0.01027006, -0.02175556,
        0.02084035, -0.00055692, -0.02506704,  0.00289898, -0.00150306,
        0.0115023 ,  0.02448574, -0.01668894, -0.02267407,  0.03224552,
        0.00503472, -0.0084677 ,  0.01429117, -0.02068722,  0.01284038,
        0.00920922, -0.01429983,  0.03295572, -0.02534479, -0.01766404,
       -0.01001931, -0.0134629 , -0.00150519, -0.01422342,  0.00606608,
        0.00314223,  0.02383385, -0.00205521,  0.02930146, -0.02464189,
        0.01791523,  0.00040434, -0.007031  ,  0.01967832, -0.01

In [ ]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(buyel_vector, Matrix_vector)

0.5739234

In [ ]:
Braveheart = movie_to_idx['Braveheart (1995)']
Braveheart_vector = als_model.item_factors[Braveheart]
np.dot(buyel_vector, Braveheart_vector)

0.23863949

In [ ]:
print (movies.loc[movies['title'] == 'Braveheart (1995)'])

     movie_id              title             genre
108       110  Braveheart (1995)  Action|Drama|War


In [ ]:
#AlternatingLeastSquares 클래스 
#similar_items 메서드를 통하여 비슷한 영화 서치
favorite_movie = 'Matrix, The (1999)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(124, 1.0),
 (92, 0.77656054),
 (62, 0.67102724),
 (200, 0.5675946),
 (141, 0.54841626),
 (145, 0.5481634),
 (107, 0.51591516),
 (375, 0.48792705),
 (175, 0.45036307),
 (317, 0.42505172),
 (75, 0.4222443),
 (44, 0.39126346),
 (117, 0.36725572),
 (193, 0.3633525),
 (479, 0.34451026)]

In [ ]:
 #artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Matrix, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Terminator, The (1984)',
 'Fugitive, The (1993)',
 'Fifth Element, The (1997)',
 'Jurassic Park (1993)',
 'Face/Off (1997)',
 'Men in Black (1997)',
 'Twelve Monkeys (1995)',
 'Hunt for Red October, The (1990)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Alien (1979)',
 'Contact (1997)']

<br>

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [ ]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [ ]:
get_similar_movie('Matrix, The (1999)')

['Matrix, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Terminator, The (1984)',
 'Fugitive, The (1993)',
 'Fifth Element, The (1997)',
 'Jurassic Park (1993)',
 'Face/Off (1997)',
 'Men in Black (1997)',
 'Twelve Monkeys (1995)']

In [ ]:
user = user_to_idx['buyel']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(117, 0.611336),
 (44, 0.57017756),
 (92, 0.5324728),
 (60, 0.51625264),
 (107, 0.488379),
 (22, 0.42072985),
 (50, 0.37725553),
 (160, 0.31556457),
 (4, 0.31548274),
 (322, 0.3029769),
 (62, 0.28613216),
 (670, 0.27534655),
 (87, 0.27498633),
 (99, 0.26587522),
 (200, 0.26205027),
 (5, 0.25703767),
 (317, 0.25694093),
 (120, 0.25455558),
 (110, 0.24128345),
 (141, 0.23880862)]

In [ ]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Terminator 2: Judgment Day (1991)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Jurassic Park (1993)',
 'Back to the Future (1985)',
 'Toy Story 2 (1999)',
 'Forrest Gump (1994)',
 "Bug's Life, A (1998)",
 'Babe (1995)',
 'Total Recall (1990)',
 'Galaxy Quest (1999)',
 'Braveheart (1995)',
 'American Beauty (1999)',
 'Terminator, The (1984)',
 'Princess Bride, The (1987)',
 'Twelve Monkeys (1995)',
 'Raiders of the Lost Ark (1981)',
 'Groundhog Day (1993)',
 'Fugitive, The (1993)']

In [ ]:
# 추천에 기여한 정도
Xman = movie_to_idx['X-Men (2000)']
explain = als_model.explain(user, csr_data, itemid=Xman)

In [ ]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Matrix, The (1999)', 0.1123100775760715),
 ('Star Wars: Episode VI - Return of the Jedi (1983)', 0.06607174087700818),
 ('Men in Black (1997)', 0.04374438838721238),
 ('Toy Story (1995)', -0.005706328812545284),
 ('E.T. the Extra-Terrestrial (1982)', -0.02411263224834979)]

<br>

## ✔️회고 및 결론
---

#### ▶️ 이번 프로젝트에서 어려웠던 점

![20220626003535](https://user-images.githubusercontent.com/100528803/175784922-448c4b34-1470-4b38-a1d1-b3f0b1f3fa5b.png)

LMS와 상이했던 에러들... 정말 LMS와 colab에서 무슨 환경에 어떻게 적용해야하는지 모호했던 에러들을 하나씩 진행한다는게 쉽지 않았다. 오죽하면 LMS에서 다시 돌아가서 돌려보기까지 했을까 싶을 정도로 여기서 제일 시행착오를 많이 겪었다.

<br>

#### ▶️ 프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점

유저에 대한 선호도를 벡터로 직접 확인할 수 있다는 점이 신기하면서도 묘했는데, 앞서 임베딩에서의 벡터를 계산하면서 단어들끼리의 상관관계를 숫자로 표현할 수 있다는 사실이랑 매칭이 됐다. 어떻게 보면 수치화 할 수 없다고 생각했던 사람의 감성에 관련된 부분들을 실제 숫자로 치환할 수 있다는 점은 기묘하면서도 자꾸만 이게 정확한 수치일지 고민하게 만든다.

<br>


#### ▶️ 루브릭 평가 지표를 맞추기 위해 시도한 것들

이번 프로젝트에서 루브릭의 기준은 아래와 같았다.

|       |                                    **< 평가문항 >**                                   |                                                              **< 상세기준 >**                                                             |
|-------|:---------------------------------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------|
| **1** | CSR matrix가 정상적으로 만들어졌다.                                 | 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.<br>                                                        |
| **2** | MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다. | 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.                               |
| **3** | 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.                                    | MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다. |

![20220626025116](https://user-images.githubusercontent.com/100528803/175785036-e7d2890c-1510-4b70-9997-3febfd4fcef5.png)

일단 선호하는 영화들을 먼저 뽑아낸 뒤에,

![20220626025141](https://user-images.githubusercontent.com/100528803/175785037-8bd37c88-e696-4010-9198-e0b699a7db20.png)

비슷한 영화들을 꼽아내고,

![20220626025157](https://user-images.githubusercontent.com/100528803/175785038-1da82b4e-1daf-4c37-9032-e3f2b1dcd3a1.png)

그로 인해 추천시스템을 통해서 엇비슷한 영화들을 뽑아낸다는 사실이 신기했다. 특히 스타워즈의 경우 가장 연관이 깊은 시리즈 5, 3, 1을 각기 뽑아냄과 동시에 back to the Future같은 SF 관련 영화들을 뽑아낸다는 점에서 어느정도 연관성이 두드러져 보였고, Toy Story의 경우 2버전을 뽑아내고, babe나 bug's life같은 3D 애니들까지 두루 뽑아냈다는 점에서 유사성을 벡터로 구현해냈다는걸 확인할 수 있었다. 결론적으로 추천시스템이 제대로 작용했다는게 신기하다.

<br>

#### ▶️ 자기 다짐

사실 내가 정말 찾고 싶었던 영화들은 리스트에 없었기 때문에 대체적으로 다른 기수분들이 썼던 데이터 중에서 인기있는 시리즈들 위주로 가져다가 많이 썼기 때문에 진짜 내 취향을 제대로 찾지 않았다는 점은 조금 아쉽다(실제로 내가 좋아하는 영화들은 이 리스트에 원래 없었다.....)

<br>

## ✔️Reference(참고자료)
---
* https://www.kaggle.com/code/alsojmc/movie-recommender-systems/notebook
* https://velog.io/@skarb4788/%EB%94%A5%EB%9F%AC%EB%8B%9D-%EC%98%81%ED%99%94-%EC%B6%94%EC%B2%9C-%EC%8B%9C%EC%8A%A4%ED%85%9C
* https://diane-space.tistory.com/90
* https://ddongwon.tistory.com/115